In [1]:
from load_matlab_elmannet import *
import numpy as np
# import gym
import time
import os
from PIL import Image

In [2]:
y_min, y_max, x_min, x_max = 25, 195, 20, 140
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [16]:
sleep_time = 0.05
model_path = "../2-elman_train/zigzag_epoch5000H200SigSig.mat"

match = 45
data_path = f"../1-generate/data/match_" + str(match) + "/"

In [27]:
ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
}

In [29]:
"""ACTIONS = {
    "right": 2,
    "left": 3,
}"""

'ACTIONS = {\n    "right": 2,\n    "left": 3,\n}'

In [30]:
ACTIONS_LIST = list(ACTIONS.values())

In [31]:
def load_npz(m):
    path = data_path + "npz/"

    actions = np.load(path + 'actions.npz')
    lifes = np.load(path + 'lifes.npz')
    frames = np.load(path + 'frames.npz')
    rewards = np.load(path + 'rewards.npz')

    arr_actions = actions.f.arr_0
    arr_lifes = lifes.f.arr_0
    arr_frames = frames.f.arr_0
    arr_rewards = rewards.f.arr_0

    print("Successfully loaded NPZ.")

    return arr_actions.shape[0], arr_frames, arr_actions, arr_rewards, arr_lifes

In [32]:
num_of_frames, frames, actions, rewards, lifes = load_npz(match)

Successfully loaded NPZ.


In [33]:
frames = frames.reshape(len(frames), -1)
frames.shape

(4500, 20400)

In [34]:
def prepare_action_data(action):

    new_action = np.zeros((1, len(ACTIONS)), dtype=int) 

    new_action[0, ACTIONS_LIST.index(action)] = 1

    return new_action

In [35]:
actions

array([0, 0, 0, ..., 3, 0, 0], dtype=int64)

In [36]:
targets = [prepare_action_data(actions[i])[0] for i in range(len(actions))]

In [37]:
targets = np.array(targets)

In [38]:
model = LoadElman(model_path)

U shape: (200, 20400)
W shape: (200, 200)
V shape: (9, 200)
bh shape: (200,)
bo shape: (9,)


In [39]:
model.reset_prev_s()

print_class = False

for i in range(len(frames)):
    if(print_class):
        print(i+1, np.argmax(targets[i]), "->", np.argmax(model.predict_step(frames[i]/255)))
    else:
        print(i+1, targets[i], "->", model.predict_step(frames[i]/255))

1 [1 0 0 0 0 0 0 0 0] -> [7.94791538e-01 2.52276603e-01 5.78402819e-02 2.52185585e-02
 1.74230587e-03 5.24283462e-03 2.35760600e-04 2.59279217e-03
 1.08176534e-03]
2 [1 0 0 0 0 0 0 0 0] -> [6.54503295e-01 4.31268378e-01 5.80012494e-02 2.99148506e-02
 2.17021634e-03 3.86315325e-03 7.75063046e-05 2.39249105e-03
 1.46953817e-03]
3 [1 0 0 0 0 0 0 0 0] -> [6.61806826e-01 2.97863546e-01 6.50019240e-02 2.29197880e-02
 1.30714603e-03 2.97497647e-03 1.42171667e-04 2.70371560e-03
 1.18771575e-03]
4 [1 0 0 0 0 0 0 0 0] -> [6.05643331e-01 4.08618161e-01 4.76257090e-02 2.07450638e-02
 1.59534105e-03 4.33010773e-03 1.54428616e-04 2.88995549e-03
 9.63213930e-04]
5 [1 0 0 0 0 0 0 0 0] -> [6.25993943e-01 4.40836690e-01 5.95505905e-02 2.59524559e-02
 3.00945695e-03 3.75893520e-03 6.53006485e-05 3.46287358e-03
 1.68017786e-03]
6 [1 0 0 0 0 0 0 0 0] -> [6.23496945e-01 4.66548245e-01 6.64611937e-02 2.45700413e-02
 3.22845238e-03 3.71684991e-03 6.06498271e-05 3.40642346e-03
 1.70985906e-03]
7 [1 0 0 0 0 0 0

92 [0 1 0 0 0 0 0 0 0] -> [2.41340708e-01 8.30230036e-01 2.95232513e-02 8.91416409e-02
 1.35272252e-02 3.82243907e-03 7.45404305e-05 2.02674777e-03
 7.56027731e-04]
93 [0 1 0 0 0 0 0 0 0] -> [1.74531458e-01 8.72479466e-01 4.29922607e-02 5.33119412e-02
 1.60418674e-02 5.60588659e-03 6.09078553e-05 1.56680174e-03
 8.76163235e-04]
94 [0 1 0 0 0 0 0 0 0] -> [9.94826877e-02 8.91722498e-01 2.00151709e-02 3.21666794e-02
 1.03263311e-02 3.37783727e-03 7.79526109e-05 1.64044225e-03
 1.38920748e-03]
95 [0 1 0 0 0 0 0 0 0] -> [1.46863726e-01 9.33528419e-01 1.95420273e-02 2.72828495e-02
 6.47492429e-03 2.74466128e-03 1.54834205e-04 1.56206604e-03
 1.50225168e-03]
96 [0 1 0 0 0 0 0 0 0] -> [2.30641293e-01 8.52886625e-01 2.15352778e-02 2.26554195e-02
 9.47787362e-03 3.39986213e-03 1.91329469e-04 3.46516876e-03
 1.31686674e-03]
97 [0 1 0 0 0 0 0 0 0] -> [1.01026166e-01 9.53164277e-01 1.86842395e-02 4.21578210e-02
 6.74796904e-03 4.13695776e-03 8.60165679e-05 2.26846511e-03
 7.26203424e-04]
98 [0 1 0 

185 [1 0 0 0 0 0 0 0 0] -> [4.42172772e-01 8.75772762e-02 6.52186071e-02 1.49679121e-01
 2.00843070e-03 5.53710340e-03 7.61000111e-05 1.11377086e-03
 9.04239183e-04]
186 [1 0 0 0 0 0 0 0 0] -> [5.20946551e-01 6.64778125e-02 5.54135212e-02 1.08458801e-01
 1.88927235e-03 6.51691577e-03 9.91787496e-05 1.39009639e-03
 7.31869996e-04]
187 [0 0 0 1 0 0 0 0 0] -> [4.98520709e-01 9.36831379e-02 8.40702923e-02 1.19636605e-01
 2.37193964e-03 7.26164067e-03 6.66222457e-05 9.34306995e-04
 6.58363060e-04]
188 [0 0 0 1 0 0 0 0 0] -> [4.44111273e-01 1.24781616e-01 4.15950343e-02 2.20194222e-01
 5.12459660e-03 6.44308630e-03 7.33167321e-05 1.77796302e-03
 3.61299991e-04]
189 [1 0 0 0 0 0 0 0 0] -> [8.65339967e-01 2.63857607e-02 2.77360573e-02 7.37352096e-02
 1.59820395e-03 2.99126857e-03 2.27401100e-04 2.53697873e-03
 5.59435282e-04]
190 [1 0 0 0 0 0 0 0 0] -> [8.64540672e-01 4.73238557e-02 4.55978294e-02 6.34586956e-02
 1.85638322e-03 4.49245759e-03 1.47029901e-04 1.25372479e-03
 2.86600675e-04]
191 

279 [1 0 0 0 0 0 0 0 0] -> [5.83995107e-01 1.32545979e-01 1.26951277e-01 2.98534449e-01
 2.73480388e-03 6.09027552e-03 5.21312844e-05 1.02753818e-03
 8.82959259e-04]
280 [1 0 0 0 0 0 0 0 0] -> [7.29258655e-01 1.53513894e-01 8.07947393e-02 2.35725119e-01
 1.77341700e-03 2.89185723e-03 1.04242037e-04 1.60135378e-03
 8.42092778e-04]
281 [1 0 0 0 0 0 0 0 0] -> [8.03883040e-01 3.86479173e-02 4.44034502e-02 1.46124018e-01
 7.66607695e-04 1.10785945e-02 2.33104673e-04 1.66987771e-03
 6.55512840e-04]
282 [1 0 0 0 0 0 0 0 0] -> [6.98210503e-01 8.05079662e-02 9.26503817e-02 1.46955149e-01
 1.84149191e-03 5.50697523e-03 1.12541470e-04 1.76986339e-03
 8.63736310e-04]
283 [1 0 0 0 0 0 0 0 0] -> [6.03848526e-01 6.22355538e-02 8.65039018e-02 2.24951560e-01
 1.46146634e-03 4.01036853e-03 5.56801773e-05 3.28126672e-03
 1.26897584e-03]
284 [1 0 0 0 0 0 0 0 0] -> [5.66957816e-01 3.95289986e-02 7.63403277e-02 1.07163964e-01
 3.09343938e-03 8.22045125e-03 7.60141233e-05 5.46123304e-03
 8.32864843e-04]
285 

376 [1 0 0 0 0 0 0 0 0] -> [6.75637030e-01 1.41447582e-01 8.28632719e-02 3.33142316e-01
 1.57853545e-03 2.97606426e-04 2.32785630e-05 5.00679835e-04
 5.62255047e-04]
377 [0 0 0 1 0 0 0 0 0] -> [6.49898590e-01 8.67825209e-02 8.80429597e-02 4.07929262e-01
 2.53812694e-03 6.10274510e-04 1.81852927e-05 8.18917898e-04
 3.84964067e-04]
378 [0 0 0 1 0 0 0 0 0] -> [7.22692662e-01 8.19446048e-02 1.60617111e-01 1.64549766e-01
 1.89902340e-03 9.38587754e-04 1.16853979e-05 2.56082472e-04
 4.63447707e-04]
379 [1 0 0 0 0 0 0 0 0] -> [4.92014918e-01 1.19078930e-01 2.23235411e-01 1.10558656e-01
 3.13345455e-03 1.01060035e-03 1.12343881e-05 3.36285825e-04
 4.56110101e-04]
380 [1 0 0 0 0 0 0 0 0] -> [5.21068756e-01 1.09528700e-01 1.91537274e-01 1.07855191e-01
 1.37221452e-03 8.62407195e-04 1.61923789e-05 3.10016912e-04
 5.37276862e-04]
381 [1 0 0 0 0 0 0 0 0] -> [4.77409414e-01 9.00740290e-02 1.69449948e-01 1.07770385e-01
 1.56351675e-03 7.89245019e-04 9.58155367e-06 2.44007086e-04
 4.87759747e-04]
382 

...

4094 [0 0 1 0 0 0 0 0 0] -> [0.42204583 0.03189008 0.49200561 0.2174308  0.0153421  0.00726847
 0.00160432 0.01574425 0.03411357]
4095 [0 0 1 0 0 0 0 0 0] -> [0.55092352 0.0138047  0.45430511 0.11828655 0.01547864 0.00751865
 0.0019958  0.04164739 0.04411339]
4096 [1 0 0 0 0 0 0 0 0] -> [0.39781983 0.01113345 0.51585903 0.06140488 0.03796239 0.01030123
 0.00254211 0.03668849 0.05772328]
4097 [1 0 0 0 0 0 0 0 0] -> [0.32675838 0.01619278 0.45445483 0.10011134 0.03833738 0.00915438
 0.00109386 0.02048063 0.03627939]
4098 [1 0 0 0 0 0 0 0 0] -> [0.62477349 0.00838161 0.48044264 0.08409589 0.01526431 0.00694942
 0.00164905 0.03695247 0.10973195]
4099 [1 0 0 0 0 0 0 0 0] -> [0.2090567  0.01449641 0.63862431 0.11908889 0.02557894 0.01199525
 0.0019497  0.04283652 0.04754828]
4100 [1 0 0 0 0 0 0 0 0] -> [0.5255536  0.01181391 0.57683569 0.16628137 0.00972644 0.00946955
 0.00227602 0.05644305 0.05768363]
4101 [0 0 1 0 0 0 0 0 0] -> [0.49942498 0.0099118  0.57029271 0.17961047 0.01281118 0.0094

4187 [1 0 0 0 0 0 0 0 0] -> [0.53598839 0.00942821 0.69261349 0.32975806 0.0201652  0.01038936
 0.00099227 0.02463572 0.01764205]
4188 [0 0 0 1 0 0 0 0 0] -> [0.27984864 0.01995207 0.64443858 0.37565933 0.03951633 0.0095045
 0.00069238 0.0158568  0.02728924]
4189 [0 0 0 1 0 0 0 0 0] -> [4.35496863e-01 1.93184026e-02 8.03532626e-01 3.35744978e-01
 5.50101399e-02 1.10534222e-02 7.32665306e-04 2.17894764e-02
 1.50398377e-02]
4190 [1 0 0 0 0 0 0 0 0] -> [0.4757233  0.01833445 0.60824014 0.15381506 0.04404338 0.01289478
 0.00083115 0.01967307 0.03223244]
4191 [1 0 0 0 0 0 0 0 0] -> [0.45539    0.01018256 0.67466769 0.2394325  0.04695432 0.0103043
 0.00073344 0.02225547 0.01521322]
4192 [1 0 0 0 0 0 0 0 0] -> [0.54714808 0.01091685 0.70569938 0.14688513 0.02547975 0.01704159
 0.00128884 0.03379277 0.0230268 ]
4193 [1 0 0 0 0 0 0 0 0] -> [0.17343567 0.02839596 0.80482708 0.3023286  0.11265964 0.0114695
 0.00116445 0.03881784 0.04910175]
4194 [1 0 0 0 0 0 0 0 0] -> [0.36641178 0.01384556 0.716

4286 [0 0 1 0 0 0 0 0 0] -> [0.1149197  0.0394698  0.63532255 0.03721427 0.03893535 0.01310066
 0.00160849 0.04878759 0.05860458]
4287 [1 0 0 0 0 0 0 0 0] -> [0.28950675 0.03835511 0.64111472 0.23314443 0.06097001 0.00820481
 0.00076002 0.00860593 0.03319957]
4288 [0 0 0 1 0 0 0 0 0] -> [0.35205898 0.03732628 0.68802557 0.17283513 0.02082046 0.01784257
 0.00151603 0.01854106 0.01920308]
4289 [0 0 0 1 0 0 0 0 0] -> [0.17997155 0.02831463 0.65588361 0.04038235 0.04497175 0.01924627
 0.00134306 0.01894912 0.0267909 ]
4290 [0 0 0 1 0 0 0 0 0] -> [0.24547343 0.02279183 0.67888098 0.09577125 0.04713724 0.01239351
 0.00091864 0.02215528 0.05230753]
4291 [1 0 0 0 0 0 0 0 0] -> [2.65066496e-01 1.08420894e-01 6.95247817e-01 1.97724308e-01
 1.75586780e-02 2.33535762e-02 2.45516422e-04 1.61809185e-02
 2.54090376e-02]
4292 [1 0 0 0 0 0 0 0 0] -> [0.20406879 0.12418333 0.83049776 0.15166176 0.03723644 0.0160871
 0.00093292 0.02135223 0.07203976]
4293 [1 0 0 0 0 0 0 0 0] -> [2.35848215e-01 7.06406315

4386 [0 0 0 0 0 0 0 1 0] -> [0.42116293 0.0250722  0.760811   0.11053856 0.02706749 0.00700531
 0.0013371  0.03028479 0.03055664]
4387 [0 0 0 0 0 0 0 1 0] -> [0.36324459 0.04950475 0.84142029 0.08630901 0.07786468 0.00846343
 0.00090231 0.03176874 0.03836172]
4388 [1 0 0 0 0 0 0 0 0] -> [0.19177431 0.03413917 0.63069193 0.06736455 0.01743317 0.0066035
 0.00130395 0.0160512  0.03033747]
4389 [1 0 0 0 0 0 0 0 0] -> [0.25036184 0.02872829 0.68837671 0.06116439 0.02680038 0.00787192
 0.00092952 0.02604551 0.0338604 ]
4390 [1 0 0 0 0 0 0 0 0] -> [2.44276257e-01 4.78410151e-02 7.07048778e-01 8.65131419e-02
 4.76536288e-02 8.00806935e-03 4.50522716e-04 1.24297147e-02
 2.66386767e-02]
4391 [1 0 0 0 0 0 0 0 0] -> [0.10013499 0.02382279 0.68841826 0.06322985 0.02968765 0.01964373
 0.0007175  0.01579122 0.0104445 ]
4392 [1 0 0 0 0 0 0 0 0] -> [0.33956275 0.01316766 0.62492316 0.11302712 0.03383335 0.01892089
 0.00083679 0.01097296 0.01928917]
4393 [1 0 0 0 0 0 0 0 0] -> [0.06522051 0.03230521 0.4

4478 [0 1 0 0 0 0 0 0 0] -> [5.08387300e-01 2.25326122e-01 2.81350466e-02 3.38607559e-01
 2.93009168e-03 4.86831794e-03 4.15082444e-05 7.29731440e-03
 1.01223515e-03]
4479 [0 1 0 0 0 0 0 0 0] -> [3.65324730e-01 2.73106887e-01 2.58420257e-02 2.24048615e-01
 3.47599648e-03 5.95554604e-03 3.61013335e-05 4.23644892e-03
 6.12580567e-04]
4480 [0 1 0 0 0 0 0 0 0] -> [3.52387319e-01 3.91576745e-01 3.64768639e-02 1.88133891e-01
 1.30846566e-03 5.23017949e-03 9.87989111e-05 1.93195432e-03
 5.42441206e-04]
4481 [0 1 0 0 0 0 0 0 0] -> [1.27938808e-01 5.30110818e-01 2.93042296e-02 3.67110557e-01
 2.59101051e-03 2.89424043e-03 5.35959927e-05 1.17390523e-03
 1.07422459e-03]
4482 [0 1 0 0 0 0 0 0 0] -> [2.07286149e-01 4.35806532e-01 3.54502163e-02 1.73019217e-01
 1.78324892e-03 1.08133807e-02 9.52947613e-05 1.09155151e-03
 6.76966575e-04]
4483 [0 1 0 0 0 0 0 0 0] -> [2.42024196e-01 5.03320341e-01 2.85564060e-02 8.07566623e-02
 3.53216770e-03 1.15624730e-02 8.11302181e-05 2.22487555e-03
 8.02847051e-04

In [33]:
frames[0]/255

array([0.16078431, 0.16078431, 0.16078431, ..., 0.        , 0.        ,
       0.        ])

In [65]:
model.reset_prev_s()
predicted = []
correct = 0
wrong = 0
for i in range( len(frames) ):
    
    if np.argmax(targets[i]) == np.argmax(model.predict_step(frames[i]/255)):
        correct += 1
    else:
        wrong +=1

In [66]:
print(correct)
print(wrong)

59
61


In [17]:
#798/(798+202)

In [23]:
env = gym.make("Enduro-v0")

In [24]:
frame = env.reset()

In [25]:
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

In [26]:
frame.shape

(210, 160, 3)

In [27]:
model.reset_prev_s()

In [ ]:
while(True):
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(20400,)
    frame = frame/255
    
    print(frame)
    
    action = model.predict_step(frame)
    action = list(ACTIONS.values())[np.argmax(action, axis=0)]
    frame, reward, done, info = env.step(action)

[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.  

[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.  

[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.        ]
[0.16078431 0.16078431 0.16078431 ... 0.         0.         0.  

[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.  

[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.        ]
[0.25098039 0.25098039 0.25098039 ... 0.         0.         0.  

...

[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.  

[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.  

[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.  

[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.  

[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.        ]
[0.43529412 0.43529412 0.43529412 ... 0.         0.         0.  